# Module 4 Session 8: Genetic ancestry inference & modeling

**Author**: Shashwat Deepali Nagar

**Date**: May 31, 2024

In [ ]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(car))
library(forestmodel)
library(class)

# Categorical ancestry inferrence

Here, we'll use data from PCA to infer categorical genetic ancestry.

### **First**, we'll try supervised clustering.  How do we do that?

In [ ]:
supervised_df <- read_delim('../Data/Session10/SupervisedClusteringDF.txt', delim = '\t')

supervised_df %>% head

supervised_df %>% dim

supervised_df %>%
count(
    PopulationGroup
)

In [ ]:
supervised_df %>%
ggplot(
    aes(PC1, PC2, fill = PopulationGroup)
) +
geom_point(pch = 21, color = 'black', size = 4) +
theme_classic() +
scale_fill_manual(
    values = c(
        'African' = '#6E2594',
        'European' = '#ECD444',
        'Asian' = '#DCEDFF',
        'Unlableled' = 'grey'
    )
) +
guides(
    x = guide_axis(cap = "both"),
    y = guide_axis(cap = "both")
) +
theme(
    axis.title = element_text(color = 'black', face = 'bold', size = 15),
    axis.text = element_text(color = 'black', size = 13),
    legend.title = element_text(color = 'black', face = 'bold', size = 15)
) 

### Step 1. Let's split our data into two dataframes

In [ ]:
labeled_df <- supervised_df %>%
filter(
    PopulationGroup != 'Unlableled'
)

labeled_df %>%
count(
    PopulationGroup
)

In [ ]:
unlabeled_df <- supervised_df %>%
filter(
    PopulationGroup == 'Unlableled'
)

unlabeled_df %>%
count(
    PopulationGroup
)

Links to documentation: 
1. [knn](https://www.rdocumentation.org/packages/class/versions/7.3-22/topics/knn)

### Step 2. Let's predict class based on 10 nearest neighbors

In [ ]:
predictions <- knn(
    train = labeled_df %>% select(-PopulationGroup), 
    test = unlabeled_df %>% select(-PopulationGroup),
    cl = labeled_df$PopulationGroup, 
    k = 10
)

table(predictions)

In [ ]:
# Adding new column for supervised labels
supervised_df <- supervised_df %>%
mutate(
    SupervisedLabels = case_when(
        PopulationGroup == 'Unlableled' ~ 'African',
        .default = PopulationGroup
    )
) 

supervised_df %>%
count(SupervisedLabels)

As expected, all the unlabeled data points were classified as `African` by the knn algorithm.

### Next, let's try unsupervised categorical ancestry inference

In [ ]:
set.seed(42)
unsupervised_clusters <- kmeans(
    supervised_df %>% select(-PopulationGroup, -SupervisedLabels), 
    centers = 3, 
    iter.max = 10, 
    nstart = 10
)

supervised_df %>%
mutate(
    UnsupervisedGroups = unsupervised_clusters$cluster
) %>%
count(
    SupervisedLabels, UnsupervisedGroups
)

Here we can see that Group `1` corresponds with the `African` supervised label, `2` corresponds with 'European', and `3` corresponds with `Asian`.

# Continuous genetic ancestry inference using ADMIXTURE

We'll use Admixture for genetic ancestry inference of data from the 1000 Genomes Project

You can download `admixture` following [this link](https://dalexander.github.io/admixture/download.html).

> This tool doesn't work on Windows!  

Since this tool is routinely used in analyses, we'll take a look at what the output data look like.

We'll use data generated by the 1000 Genomes Project.  [Linked here](https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/supporting/admixture_files/)

### Loading the P file

In [ ]:
p_df <- read_delim(
    '../Data/Session10/ALL.wgs.phase3_shapeit2_filtered.20141217.maf0.05.5.P', 
    delim = ' ',
    col_names = c('Pop1', 'Pop2', 'Pop3', 'Pop4', 'Pop5')
)

p_df %>% head

p_df %>% dim

In [ ]:
q_df <- read_delim(
    '../Data/Session10/ALL.wgs.phase3_shapeit2_filtered.20141217.maf0.05.5.Q', 
    delim = ' ',
    col_names = c('Pop1', 'Pop2', 'Pop3', 'Pop4', 'Pop5')
)

q_df %>% head

q_df %>% dim

# Modeling T2D and genetic ancestry

In [ ]:
df <- read_delim('../Data/Session10/Cohort_Session10.txt', delim = '\t')

df %>% head

df %>% dim

The columns `European`, `EastAsian`, `Amerindian`, `SouthAsian`, `African`, and `NorthAfrican` are continuous genetic ancestry estimates for each of the 400,000 subjects here.

In [ ]:
df %>%
group_by(EthnicGroup) %>%
summarize(
    European = round(mean(European) * 100, 2),
    EastAsian = round(mean(EastAsian) * 100, 2),
    Amerindian = round(mean(Amerindian) * 100, 2),
    African = round(mean(African) * 100, 2),
    NorthAfrican = round(mean(NorthAfrican) * 100, 2),
    SouthAsian = round(mean(SouthAsian) * 100, 2)
)

In [ ]:
df %>%
ggplot(
    aes(PC1, PC2, fill = EthnicGroup)
) +
geom_point(pch = 21, color = 'black', size = 3) +
theme_classic() +
scale_fill_manual(
    values = c(
        'Black' = '#6E2594',
        'White' = '#ECD444',
        'Asian' = '#DCEDFF'
    )
) +
guides(
    x = guide_axis(cap = "both"),
    y = guide_axis(cap = "both")
) +
theme(
    axis.title = element_text(color = 'black', face = 'bold', size = 15),
    axis.text = element_text(color = 'black', size = 13),
    legend.title = element_text(color = 'black', face = 'bold', size = 15)
) 

In [ ]:
df %>%
ggplot(
    aes(PC1, PC2, fill = EthnicGroup)
) +
geom_point(pch = 21, color = 'black', size = 3, alpha = 0.2) +
theme_classic() +
scale_fill_manual(
    values = c(
        'Black' = '#6E2594',
        'White' = '#ECD444',
        'Asian' = '#DCEDFF'
    )
) +
guides(
    x = guide_axis(cap = "both"),
    y = guide_axis(cap = "both")
) +
theme(
    axis.title = element_text(color = 'black', face = 'bold', size = 15),
    axis.text = element_text(color = 'black', size = 13),
    legend.title = element_text(color = 'black', face = 'bold', size = 15)
) 

For the purposes of the next section, we'll only include Subject with > 90%:
1. South Asian ancestry
2. European ancestry
3. African ancestry

In [ ]:
working_df <- df %>%
filter(
    African > 0.90 | European > 0.90 | SouthAsian > 0.90
) %>%
mutate(
    GAGroup = case_when(
        African > 0.9 ~ 'African',
        European > 0.9 ~ 'European',
        SouthAsian > 0.90 ~ 'SouthAsian'
    ),
    GAGroup = factor(
        GAGroup,
        levels = c('European', 'African', 'SouthAsian')
    )
)

working_df %>% 
group_by(EthnicGroup) %>%
summarize(
    European = round(mean(European) * 100, 2),
    EastAsian = round(mean(EastAsian) * 100, 2),
    Amerindian = round(mean(Amerindian) * 100, 2),
    African = round(mean(African) * 100, 2),
    NorthAfrican = round(mean(NorthAfrican) * 100, 2),
    SouthAsian = round(mean(SouthAsian) * 100, 2)
)

In [ ]:
working_df %>%
ggplot(
    aes(PC1, PC2, fill = EthnicGroup)
) +
geom_point(pch = 21, color = 'black', size = 3, alpha = 0.2) +
theme_classic() +
scale_fill_manual(
    values = c(
        'Black' = '#6E2594',
        'White' = '#ECD444',
        'Asian' = '#DCEDFF'
    )
) +
guides(
    x = guide_axis(cap = "both"),
    y = guide_axis(cap = "both")
) +
theme(
    axis.title = element_text(color = 'black', face = 'bold', size = 15),
    axis.text = element_text(color = 'black', size = 13),
    legend.title = element_text(color = 'black', face = 'bold', size = 15)
) 

In [ ]:
working_df %>%
count(
    EthnicGroup, GAGroup
)

## Modeling T2D

In [ ]:
overall_model <- glm(
    T2D ~ scale(Age) + Sex + scale(SED) + GAGroup,
    data = working_df,
    family = 'binomial'
)

summary(overall_model)

vif(overall_model) %>%
as_tibble(rownames = 'Predictor')

forest_model(overall_model)

In [ ]:
overall_model <- glm(
    T2D ~ scale(Age) + Sex + scale(SED) * GAGroup,
    data = working_df,
    family = 'binomial'
)

summary(overall_model)

vif(overall_model) %>%
as_tibble(rownames = 'Predictor')

forest_model(overall_model)